# Oura Ring Neural Network Prediction Machine

In [332]:
import numpy
import scipy.special
import matplotlib.pyplot

In [333]:
class neuralNetwork:
    def __init__(self, inputnode, hiddennode, outputnode, learningrate):
        self.inodes = inputnode
        self.hnodes = hiddennode
        self.onodes = outputnode
        self.lr = learningrate
        
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    
    def train(self, input_list, target_list):
        inputs = numpy.array([input_list]).T
        targets = numpy.array([target_list]).T
        
        # Forward pass
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # Error Calculation
        output_error = targets - final_outputs
        hidden_error = numpy.dot(self.who.T, output_error)
        
        # Adjusting Weights
        self.who += self.lr * numpy.dot((output_error * final_outputs * (1 - final_outputs)), numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_error * hidden_outputs* (1- hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    
    def query(self, inputs_list):
        inputs = numpy.array([inputs_list]).T
        
        # Forward pass
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Network initialisieren

In [334]:
inputnodes = 29
hiddennodes = 200
outputnodes = 9
learningrate = 0.2

n = neuralNetwork(inputnodes, hiddennodes, outputnodes, learningrate)

## Trainingsset einladen
- len 99

In [335]:
training_data_file = open("all_data_oura_training.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

## Trainings Network

In [336]:
epoch = 50

for i in range(epoch):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 100 * 0.99) + 0.01

        # target matrix
        targets = numpy.zeros(outputnodes) + 0.01
        x = (int(all_values[0]) - 50) // 5
        targets[x] = 0.99

        n.train(inputs, targets)
        
        pass
    pass

## Test Network

In [337]:
test_data_file = open("all_data_oura_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [338]:
scorecard = []

for record in test_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 100 * 0.99) + 0.01

        outputs = n.query(inputs)
        prediction = numpy.argmax(outputs)
        
        x = (int(all_values[0]) - 50) // 5
        
        if x == prediction:
            scorecard.append(1)
        else:
            scorecard.append(0)
        
        pass

In [339]:
scorecard_average = numpy.sum(scorecard) / len(scorecard)
print(scorecard_average)
print(scorecard)

0.45454545454545453
[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
